# Microsoft Movie Analysis

**Authors:** Kevin Culver
***

## Overview

The project analyzes various movie databases to provide tangible steps for Microsoft to take as it begins to create movie content. Analysis of these databases showed that certain characteristics of a movie, such as its genre, budget, or run time produced greater profit. Analysis also revealed common trends that top-grossing films share. These findings can be used by Microsoft to narrow down decisions on what types of films to produce and invest in.

## Business Problem

Creating a successful movie takes time, money, and risk. However, a company, like Microsoft, cannot predict how the movie will be received by it's audience. Therefore, the point of my analysis was to find what qualities are shared by high grossing films. to provide insight into what characteristics of movies have the greatest likelihood of monetary success. Using this data, I provide recommendations... 
***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***

## Data Understanding

The data used in this project comes from three online movie databases - iMDB, The Numbers, and Box Office Mojo. Within these databases, each movie is identified by its title or movie id. When combined, these databases provide comprehensive information for each movie, including its rating, release date, budget, gross (both domestic and worldwide), genre and other characteristics. 


### Importing Relevant Modules and Datasets

In [23]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

%matplotlib inline

In [3]:
#iMDB datasets
imdb_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')
imdb_titles = pd.read_csv('zippedData/imdb.title.basics.csv.gz')

#The Numbers (movie budget dataset)
budget = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

### iMDB and The Numbers Data

#### iMDB Data
Two datasets imported from iMDB are: *imdb_ratings* and *imdb_titles*. 

The *imdb_ratings* database, provides the average ratings and number of votes for each movies.  

The second database, *imdb_titles*, provides more info on each movie including it's title, runtime, start year, and genre. 


These two datasets can be easily joined since they share the unique movie identifier 'tconst'. In the cells below, the iMDB databases are joined using an inner join. 

In [6]:
#setting indexes to 'tconst' for each dataset to prep for inner join
imdb_titles.set_index('tconst', inplace=True)
imdb_ratings.set_index('tconst', inplace=True)

In [14]:
#joining dfs using inner join to get iMDB movies that contain both titles and ratings
imdb_titles_ratings = imdb_titles.join(imdb_ratings, how='inner')

imdb_titles_ratings.head(2)


,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
tconst,,,,,,,
tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43


At this point in the analysis, there are close to 74,000 entries. The start year's range from 2010-2019, but as we'll see this is column does not represent the movie's actual release date. 

#### The Numbers Data:

'The Numbers' dataset includes information on each movie's release date, title, budget, and total gross(both domestic and worldwide). The dataset contains 5782 entries.

In [21]:
budget.head(2)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


#### Merging Data Tables Together

The two tables are joined using movie title to create one movie data frame. Before cleaning the data frame has 2638 entries. With movies' release date ranging from 1940-2019.

In [20]:
#merging the two tables (imdb + budget) using movie titles
movies_df = pd.merge(imdb_titles_ratings, budget, right_on='movie', left_on='original_title')
movies_df.head(2)

,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,Foodfight!,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,8248,26,"Dec 31, 2012",Foodfight!,"$45,000,000",$0,"$73,706"
1,The Overnight,The Overnight,2010,88.0,NaN,7.5,24,21,"Jun 19, 2015",The Overnight,"$200,000","$1,109,808","$1,165,996"


### Creating Top 2021 Movies Table Using Webscraping

Since the data from the tables above only includes movies up to 2019, I wanted to get more recent data. To do so, I webscraped data from the website boxofficemojo.com to from a table with the top 200 movies from 2021.

The data includes the movie's current rank, title, gross, release date, and studio.

In [24]:
html_page = requests.get("https://www.boxofficemojo.com/year/2021/?ref_=bo_lnav_hm_shrt")

soup = BeautifulSoup(html_page.content, 'html.parser')

In [25]:
#finding relevant division and creating container
table = soup.find('div', id='table')

#seperates all the relevant rows with movie data from table headers
table_rows = table.findAll('tr')[1:]

#length outputs 200 items, which corresponds to the 200 items on the website
len(table_rows)

200

In [31]:
#finding rank
rank = table_rows[0].find('td', class_="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank mojo-sort-column").text

#finding title
title = table_rows[0].find('td', class_= "a-text-left mojo-field-type-release mojo-cell-wide").text

#finding amount grossed
gross = table_rows[0].find('td', class_ = "a-text-right mojo-field-type-money mojo-estimatable").text

#finding release date
release_date = table_rows[0].find('td', class_ = "a-text-left mojo-field-type-date a-nowrap").text

#finding studio
studio =  table_rows[0].find('td',class_ = "a-text-left mojo-field-type-studio").text.strip()


In [32]:
#Function to retrieve each row/movie from website
def get_movie_data(movie_table):
    movie_data = []
    for row in movie_table:
        rank = row.find('td', class_="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank mojo-sort-column").text
        title = row.find('td', class_= "a-text-left mojo-field-type-release mojo-cell-wide").text
        gross = row.find('td', class_ = "a-text-right mojo-field-type-money mojo-estimatable").text
        release_date = row.find('td', class_ = "a-text-left mojo-field-type-date a-nowrap").text
        studio = row.find('td',class_ = "a-text-left mojo-field-type-studio").text.strip()
        
        movie_data.append({'Rank': rank, 'Title': title, 'Gross': gross, 
                           'Release Date': release_date, 'Studio': studio})
    return movie_data

In [36]:
movie_data = get_movie_data(table_rows)
rankings_2021_df = pd.DataFrame(movie_data)

rankings_2021_df.head(2)

,Rank,Title,Gross,Release Date,Studio
0,1,Shang-Chi and the Legend of the Ten Rings,"$223,361,907",Sep 3,Walt Disney Studios Motion Pictures
1,2,Venom: Let There Be Carnage,"$193,717,635",Oct 1,Sony Pictures Entertainment (SPE)


## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [6]:
# Here you run your code to clean the data

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [ ]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***